In [1]:
# !pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers==4.31.0 accelerate
# !pip install "nvidia-modelopt[all]" -U --extra-index-url https://pypi.nvidia.com
!pip install torchinfo

Defaulting to user installation because normal site-packages is not writeable


In [10]:
# !pip install torch==2.5.1

  Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
Using cached triton-3.1.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.21.0 requires torch==2.6.0, but you have torch 2.5.1 which is incompatible.


In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

PyTorch version: 2.6.0+cu124
CUDA available: True


In [2]:
!echo $LOCAL

/local


In [3]:
!echo $HF_HOME
# !export HF_HOME=""

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from torchinfo import summary

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_ID = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tok = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [5]:
import copy, modelopt.torch.quantization as mtq
cfg = copy.deepcopy(mtq.INT4_AWQ_CFG)
cfg["quant_cfg"]["*lm_head*"] = {"enable": False}   # keep logits full‑precision :contentReference[oaicite:2]{index=2}

In [6]:
from datasets import load_dataset, Dataset
import torch, random

raw = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
texts = random.sample(list(raw["text"]), k=128)       # ~50 k tokens
def collate(batch):
    ids = tok(batch, return_tensors="pt", padding=True, truncation=True,
              max_length=2048)["input_ids"].to(model.device)
    return ids
calib_loader = torch.utils.data.DataLoader(texts, batch_size=2, collate_fn=collate)


In [7]:
seq_len  = 128
batch    = torch.randint(
              0, model.config.vocab_size,        # token IDs
              (1, seq_len),                       # (B, L)
              dtype=torch.long,
              device=device)
summary(model, input_data=batch, col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])

Layer (type:depth-idx)                        Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Qwen2ForCausalLM                              [1, 128]                  --                        --                        --                        --
├─Qwen2Model: 1-1                             --                        --                        --                        --                        --
│    └─Embedding: 2-1                         [1, 128]                  [1, 128, 1536]            233,373,696               --                        233,373,696
│    └─Qwen2RotaryEmbedding: 2-2              [1, 128, 1536]            [1, 128, 128]             --                        --                        --
│    └─ModuleList: 2-3                        --                        --                        --                        --                        --
│    │    └─Qwen2DecoderLayer: 3-1            [1, 128, 1536]      

In [ ]:
import modelopt.torch.quantization as mtq

model_pre_q = copy.deepcopy(model).to(device)

def forward_loop(m, max_batch = 1000):                 # PTQ forward pass
    for batch_idx, ids in enumerate(calib_loader):
        m(ids)
        if batch_idx >= max_batch:
            break



qmodel = mtq.quantize(model_pre_q, cfg, forward_loop)       # :contentReference[oaicite:3]{index=3}


Definition of _QuantQwen2Attention saved to /var/tmp/modelopt_asracd02.py
Successfully registered Qwen2Attention for quantization
Inserted 675 quantizers
Caching activation statistics for awq_lite...
Searching awq_lite parameters...
Loading extension modelopt_cuda_ext...


/jet/home/fxiang1/.local/lib/python3.12/site-packages/modelopt/torch/utils/cpp_extension.py:88: UserWarning: Error building extension 'modelopt_cuda_ext': [1/3] /opt/packages/cuda/v12.6.1/bin/nvcc --generate-dependencies-with-compile --dependency-output tensor_quant_gpu.cuda.o.d -DTORCH_EXTENSION_NAME=modelopt_cuda_ext -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /jet/home/fxiang1/.local/lib/python3.12/site-packages/torch/include -isystem /jet/home/fxiang1/.local/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /jet/home/fxiang1/.local/lib/python3.12/site-packages/torch/include/TH -isystem /jet/home/fxiang1/.local/lib/python3.12/site-packages/torch/include/THC -isystem /opt/packages/cuda/v12.6.1/include -isystem /jet/home/psamal/hw_envs/idl_hw4/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_B

In [ ]:
out_dir = "deepseek-qwen1.5b-awq-int4"
qmodel.save_pretrained(out_dir)
tok.save_pretrained(out_dir)


Saved ModelOpt state to deepseek-qwen1.5b-awq-int4/modelopt_state.pth


('deepseek-qwen1.5b-awq-int4/tokenizer_config.json',
 'deepseek-qwen1.5b-awq-int4/special_tokens_map.json',
 'deepseek-qwen1.5b-awq-int4/tokenizer.json')

In [ ]:
mtq.print_quant_summary(qmodel)

model.layers.0.self_attn.q_proj.input_quantizer                                  TensorQuantizer(disabled)
model.layers.0.self_attn.q_proj.output_quantizer                                 TensorQuantizer(disabled)
model.layers.0.self_attn.q_proj.weight_quantizer                                 TensorQuantizer(4 bit fake block_sizes={-1: 128, 'type': 'static'}, amax=[0.0173, 1.2783](18432) calibrator=MaxCalibrator quant)
model.layers.0.self_attn.k_proj.input_quantizer                                  TensorQuantizer(disabled)
model.layers.0.self_attn.k_proj.output_quantizer                                 TensorQuantizer(disabled)
model.layers.0.self_attn.k_proj.weight_quantizer                                 TensorQuantizer(4 bit fake block_sizes={-1: 128, 'type': 'static'}, amax=[0.0236, 0.4045](3072) calibrator=MaxCalibrator quant)
model.layers.0.self_attn.v_proj.input_quantizer                                  TensorQuantizer(disabled)
model.layers.0.self_attn.v_proj.output_quantiz

In [ ]:
seq_len  = 128
batch    = torch.randint(
              0, qmodel.config.vocab_size,        # token IDs
              (1, seq_len),                       # (B, L)
              dtype=torch.long,
              device=device)
summary(qmodel, input_data=batch, col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Qwen2ForCausalLM                                        [1, 128]                  --                        --                        --                        --
├─Qwen2Model: 1-1                                       --                        --                        --                        --                        --
│    └─Embedding: 2-1                                   [1, 128]                  [1, 128, 1536]            233,373,696               --                        233,373,696
│    └─Qwen2RotaryEmbedding: 2-2                        [1, 128, 1536]            [1, 128, 128]             --                        --                        --
│    └─ModuleList: 2-3                                  --                        --                        --                        --                        --
│    │

In [ ]:
seq_len  = 128
batch    = torch.randint(
              0, model_pre_q.config.vocab_size,        # token IDs
              (1, seq_len),                       # (B, L)
              dtype=torch.long,
              device=device)
summary(model_pre_q, input_data=batch, col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])

Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Qwen2ForCausalLM                                        [1, 128]                  --                        --                        --                        --
├─Qwen2Model: 1-1                                       --                        --                        --                        --                        --
│    └─Embedding: 2-1                                   [1, 128]                  [1, 128, 1536]            233,373,696               --                        233,373,696
│    └─Qwen2RotaryEmbedding: 2-2                        [1, 128, 1536]            [1, 128, 128]             --                        --                        --
│    └─ModuleList: 2-3                                  --                        --                        --                        --                        --
│    │

In [ ]:
mtq.compress(qmodel)